In [1]:
# Below is the modified Python code to handle 15-minute intervals, 
# incorporating solar fluctuations and aggregating results over 1 hour for billing.
# 23 April 25
import numpy as np
# System Parameters
# Loads (kWh, per 15-minute interval, assumed constant)
L = np.array([25, 12.5, 37.5, 40])

# Average Solar Production (kWh, per 15-minute interval)
S_avg = np.array([50, 15, 5, 5])

# Number of buildings and intervals
n_buildings = 4
n_intervals = 4  # 4 x 15-minute intervals = 1 hour

# Simulate solar fluctuations (±30% around average)
np.random.seed(42)  # For reproducibility
fluctuation = 0.3  # ±30%
S_intervals = np.zeros((n_intervals, n_buildings))
for t in range(n_intervals):
    for i in range(n_buildings):
        # Random variation within ±30% of average
        S_intervals[t, i] = S_avg[i] * (1 + np.random.uniform(-fluctuation, fluctuation))
        S_intervals[t, i] = round(S_intervals[t, i], 2)

# Initialize arrays to store results
E_total = np.zeros((n_buildings, n_buildings))  # Total transfers over 1 hour
M_total = np.zeros(n_buildings)  # Total MDB readings
M5_total = 0  # Total grid import

# Process each 15-minute interval
for t in range(n_intervals):
    print(f"\nInterval {t+1} (15 minutes):")
    S = S_intervals[t, :]
    print(f"Solar Production (kWh): {S}")

    # Step 1: Calculate Surplus and Deficit
    surplus_deficit = S - L
    surplus = np.zeros(n_buildings)
    deficit = np.zeros(n_buildings)
    for i in range(n_buildings):
        if surplus_deficit[i] > 0:
            surplus[i] = surplus_deficit[i]
        else:
            deficit[i] = -surplus_deficit[i]
    
    total_surplus = np.sum(surplus)
    total_deficit = np.sum(deficit)
    print(f"Surplus (kWh): {surplus}")
    print(f"Deficit (kWh): {deficit}")
    print(f"Total Surplus: {total_surplus:.2f} kWh, Total Deficit: {total_deficit:.2f} kWh")

    # Step 2: Pre-transfer MDB Readings
    M_pre = np.zeros(n_buildings)
    for i in range(n_buildings):
        M_pre[i] = S[i] - L[i]  # Negative for exporters, positive for importers
    print(f"Pre-transfer MDB (kWh): {M_pre}")

    # Step 3: Allocate Transfers
    E = np.zeros((n_buildings, n_buildings))
    if total_deficit > 0:
        deficit_shares = np.zeros(n_buildings)
        for i in range(n_buildings):
            if deficit[i] > 0:
                deficit_shares[i] = deficit[i] / total_deficit
        
        for i in range(n_buildings):
            if M_pre[i] < 0:  # Exporting building
                surplus_i = -M_pre[i]
                for j in range(n_buildings):
                    if deficit[j] > 0:
                        E[i, j] = deficit_shares[j] * surplus_i
                        E[i, j] = round(E[i, j], 2)
    
    print(f"Transfers (kWh):")
    for i in range(n_buildings):
        for j in range(n_buildings):
            if E[i, j] > 0:
                print(f"E_{i+1}->{j+1}: {E[i, j]:.2f}")

    # Step 4: Calculate MDB Meter Readings
    M = np.zeros(n_buildings)
    for i in range(n_buildings):
        received = np.sum(E[:, i])
        exported = np.sum(E[i, :])
        M[i] = L[i] - S[i] - received + exported
        M[i] = round(M[i], 2)
    
    M5 = np.sum(M)
    print(f"MDB Readings (kWh): {M}")
    print(f"Grid Import (M_5): {M5:.2f} kWh")

    # Step 5: Verify Billing with Final M_j
    total_import = np.sum(M[M > 0])
    if total_import > 0:
        import_shares = np.zeros(n_buildings)
        for j in range(n_buildings):
            if M[j] > 0:
                import_shares[j] = M[j] / total_import
        
        E_final = np.zeros((n_buildings, n_buildings))
        for i in range(n_buildings):
            if M_pre[i] < 0:
                surplus_i = -M_pre[i]
                for j in range(n_buildings):
                    if M[j] > 0:
                        E_final[i, j] = import_shares[j] * surplus_i
                        E_final[i, j] = round(E_final[i, j], 2)
    else:
        E_final = E
    
    # Accumulate results
    E_total += E_final
    M_total += M
    M5_total += M5

# Step 6: Aggregate for Hourly Billing
print("\nHourly Billing (Aggregated over 4 Intervals):")
print(f"Total Loads (kWh): {L * n_intervals}")
print(f"Total Solar Production (kWh): {np.sum(S_intervals, axis=0)}")
print(f"\nTotal Transfers (kWh):")
for i in range(n_buildings):
    for j in range(n_buildings):
        if E_total[i, j] > 0:
            print(f"E_{i+1}->{j+1}: {E_total[i, j]:.2f}")
print(f"\nTotal MDB Readings (kWh): {M_total}")
print(f"Total Grid Import (M_5): {M5_total:.2f} kWh")

print("\nBilling Outcome:")
print("Inter-building Billing:")
for j in range(n_buildings):
    if np.sum(E_total[:, j]) > 0:
        print(f"Building {j+1}:")
        for i in range(n_buildings):
            if E_total[i, j] > 0:
                print(f"  {E_total[i, j]:.2f} kWh from Building {i+1}")
for i in range(n_buildings):
    total_exported = np.sum(E_total[i, :])
    if total_exported > 0:
        print(f"Building {i+1}: Credited for {total_exported:.2f} kWh")
print("\nGrid Billing:")
for i in range(n_buildings):
    print(f"Building {i+1}: {M_total[i]:.2f} kWh")
print(f"Total Grid Import: {M5_total:.2f} kWh")


Interval 1 (15 minutes):
Solar Production (kWh): [46.24 19.06  5.7   5.3 ]
Surplus (kWh): [21.24  6.56  0.    0.  ]
Deficit (kWh): [ 0.   0.  31.8 34.7]
Total Surplus: 27.80 kWh, Total Deficit: 66.50 kWh
Pre-transfer MDB (kWh): [ 21.24   6.56 -31.8  -34.7 ]
Transfers (kWh):
E_3->3: 15.21
E_3->4: 16.59
E_4->3: 16.59
E_4->4: 18.11
MDB Readings (kWh): [-21.24  -6.56  31.8   34.7 ]
Grid Import (M_5): 38.70 kWh

Interval 2 (15 minutes):
Solar Production (kWh): [39.68 11.9   3.67  6.1 ]
Surplus (kWh): [14.68  0.    0.    0.  ]
Deficit (kWh): [ 0.    0.6  33.83 33.9 ]
Total Surplus: 14.68 kWh, Total Deficit: 68.33 kWh
Pre-transfer MDB (kWh): [ 14.68  -0.6  -33.83 -33.9 ]
Transfers (kWh):
E_2->2: 0.01
E_2->3: 0.30
E_2->4: 0.30
E_3->2: 0.30
E_3->3: 16.75
E_3->4: 16.78
E_4->2: 0.30
E_4->3: 16.78
E_4->4: 16.82
MDB Readings (kWh): [-14.68   0.6   33.83  33.9 ]
Grid Import (M_5): 53.65 kWh

Interval 3 (15 minutes):
Solar Production (kWh): [53.03 16.87  3.56  6.41]
Surplus (kWh): [28.03  4.37  0.  